In [157]:
############################################################################################################################
#############################                     convert data into df                    ##################################
############################################################################################################################


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from patsy import dmatrices
import statsmodels.api as sm


from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


# read data as framework

df = pd.read_csv('./data_integer_shift_column.csv')

# check the framework

df.head()

,production,azimuth,md (ft),tvd (ft),footage lateral length,well spacing,porpoise deviation,porpoise count,shale footage,acoustic impedance,...,youngs modulus,isip,breakdown pressure,pump rate,total number of stages,proppant volume,proppant fluid ratio,tc_number,op_number,date_sec
0,5614.947951,-32.279999,19148,6443.0,11966.0,4368.46290,6.33,12,1093,30123.20,...,30.82,4149.0,NaN,83,56,21568792.0,1.23,1,1,1519862400
1,2188.836707,-19.799999,15150,7602.0,6890.0,4714.99220,1.28,4,0,30951.61,...,29.72,5776.0,NaN,102,33,9841307.0,1.47,2,2,1404172800
2,1450.033022,-26.879999,14950,5907.0,8793.0,798.92096,2.03,6,3254,28900.25,...,30.99,4628.0,NaN,88,62,17116240.0,1.67,3,1,1533081600
3,1060.764407,-49.099998,11098,6538.0,4234.0,NaN,6.00,23,7470,32826.08,...,26.20,4582.0,NaN,100,11,3749559.0,0.77,4,1,1325376000
4,607.530385,5.560000,10549,7024.0,2972.0,2967.56300,11.87,9,3637,26740.05,...,31.18,4909.0,NaN,94,9,6690705.0,1.32,5,3,1325376000


In [158]:
############################################################################################################################
###############################                     count missing data            ##########################################
############################################################################################################################

# the percentage of missing data in df

print('-------------------------------')

print('missing percentage (%):')

print('-------------------------------')
df.isna().sum()/len(df)*100

-------------------------------
missing percentage (%):
-------------------------------


production                 0.0
azimuth                    5.5
md (ft)                    0.0
tvd (ft)                   2.0
footage lateral length     0.0
well spacing              15.6
porpoise deviation         0.0
porpoise count             0.0
shale footage              0.0
acoustic impedance         0.0
log permeability           0.0
porosity                  11.9
poisson ratio              0.0
water saturation          57.7
toc                        2.1
vcl                        0.0
p-velocity                 0.0
s-velocity                 0.0
youngs modulus             1.9
isip                       7.7
breakdown pressure        74.4
pump rate                  0.0
total number of stages     0.0
proppant volume           13.2
proppant fluid ratio       0.0
tc_number                  0.0
op_number                  0.0
date_sec                   0.0
dtype: float64

In [159]:
#########################################################################################################################

#As aboserved
    ## ~75% data in 'breakdown pressure'
    ## ~58% data in 'water saturation'
    ## ~15% data in 'well spacing'
    ## ~13% data in 'proppant volume'
    ## ~12% data in 'porosity'
    ## ~8% data in 'isip'
    ## ~5% data in 'azimuth'
    ## ~2% data in 'tvd'
    ## ~2% data in 'toc'
    ## ~2% data in 'youngs modulus'
    
    
#############################################   using KNN to predict the missing data   #####################################################


##############################    scale the data before prediction

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_sc = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
# df_sc.head()


##############################    KNN prediction

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df_sc_knn = pd.DataFrame(imputer.fit_transform(df_sc),columns = df_sc.columns)

df_sc_knn.isna().sum().sum()

0

In [160]:
############################################################################################################################
#################################################    find critical variables   #############################################
############################################################################################################################


################################################# use all varables to predict, find adj R2

import statsmodels.api as sm

x = df_sc_knn.drop(['production'], axis=1)
y = df_sc_knn['production']
                
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:             production   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.539
Method:                 Least Squares   F-statistic:                     44.25
Date:                Tue, 27 Oct 2020   Prob (F-statistic):          1.01e-148
Time:                        02:51:36   Log-Likelihood:                 856.44
No. Observations:                1000   AIC:                            -1657.
Df Residuals:                     972   BIC:                            -1519.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0

In [161]:
#########################################      delete the attribute with too large P value 

### according to reference, if |P value| < 0.05, the feature is significant
### to be conservative, we choose |P value| < 0.2, and delete the value =>0.05

p_values = est2.summary2().tables[1]['P>|t|']

rstl_p = pd.DataFrame(p_values)

rstl_p.loc[(rstl_p['P>|t|']>=0.2)]

,P>|t|
azimuth,0.654871
md (ft),0.314910
footage lateral length,0.566755
porpoise deviation,0.973015
porpoise count,0.459458
shale footage,0.544183
acoustic impedance,0.242992
water saturation,0.285633
toc,0.919138
vcl,0.598908


In [162]:
###################################   calculate the variance inflation factor (VIF)   #######################################


## Multicollinearity occurs when two or more predictors in the model are correlated
## it provide redundant information about the response
## Multicollinearity was measured by variance inflation factors (VIF) and tolerance
## VIF >10 is not acceptable

X = add_constant(df_sc_knn)


# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                          for i in range(len(X.columns))] 

############################## list the attributes with 'VIF'>10

rslt_df = vif_data[vif_data['VIF']>10] 
  
print('\feature with VIF>10 :\n', rslt_df) 

eature with VIF>10 :
                    feature         VIF
0                    const  360.121970
3                  md (ft)   58.612310
5   footage lateral length   58.064442
19          youngs modulus   10.370004


In [163]:
#############  Accroding to P value, we can delete

                # azimuth
                # md (ft)
                # footage lateral length
                # porpoise deviation
                # porpoise count
                # acoustic impedance
                # shale footage
                # water saturation
                # toc
                # vcl
                # proppant fluid ratio
                # tc_number
                
#############  Accroding to VIF value, we can delete

                # md (ft)
                # footage lateral length 
                # youngs modulus
            
################################
# we will delete all attributes recommended by P value and VIF
# except, 'young modulus', as the reference has reported the correlation to production 

In [164]:
############################## they are 'md(ft)', 'footage lateral length', and 'youngs modulus'

# df_sc_drop = df_sc_knn.drop(['md (ft)', 'footage lateral length'], axis=1)

df_sc_drop = df_sc_knn.drop(['azimuth',
                             'md (ft)',
                             'footage lateral length',
                             'porpoise deviation',
                             'porpoise count',
                             'acoustic impedance',
                             'shale footage',
                             'water saturation', 
                             'toc',
                             'vcl',
                             'proppant fluid ratio',
                             'tc_number'], axis=1)

In [165]:
################################################# use remaining varables to predict, find adj R2

x = df_sc_drop.drop(['production'], axis=1)
y = df_sc_drop['production']
                
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:             production   R-squared:                       0.548
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     79.46
Date:                Tue, 27 Oct 2020   Prob (F-statistic):          9.64e-158
Time:                        02:51:36   Log-Likelihood:                 852.42
No. Observations:                1000   AIC:                            -1673.
Df Residuals:                     984   BIC:                            -1594.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0

In [166]:
#################################
## as observed the adjusted R square is improved by deleting the variables

In [167]:
################################# correlation ##########################################

## the coefficient returns a value between -1 and 1 
## the value represents the limits of correlation from a full negative correlation to a full positive correlation
## according to reference, |value| <0.1 means weak correlation
## to be conservative, we choose |value| <0.01

df_sc_drop_corr = df_sc_drop.corr(method ='pearson').round(2) 
df_sc_drop_corr

# list the correlation between featurs and production was in the range of -0.2 to 0.2

rslt_corr = df_sc_drop_corr.loc[(df_sc_drop_corr['production']>=-0.2) & (df_sc_drop_corr['production']<=0.2)]

rslt_corr['production']

tvd (ft)              0.18
well spacing          0.01
log permeability      0.02
porosity              0.03
poisson ratio        -0.14
s-velocity           -0.20
isip                  0.16
breakdown pressure    0.12
pump rate             0.14
op_number            -0.13
Name: production, dtype: float64

In [177]:
#################################

#############  Accroding to P value, we can delete 
######because (|p| value of  and  is larger than 0.05)

                # isip
                # pump rate

                
#############  Accroding to correlation value, we can delete

                # tvd (ft)
                # well spacing
                # log permeability
                # porosity 
                # poisson ratio
                # s-velocity
                # isip
                # breakdown pressure
                # pump rate
                # op_number
                
################################
# we will delete all attributes recommended by P value and VIF
# except, 'log permeability' and 'porosity' as the reference has reported the correlation to production 

############################## drop atrributes

df_sc_drop_2 = df_sc_drop.drop([#'tvd (ft)',
                                 #'well spacing',
                                 #'poisson ratio',
                                 #'s-velocity',
                                 'isip',
                                 'breakdown pressure',
                                 'pump rate',
                                 #'op_number'
                                    ], axis=1)

###############################  again, use remaining varables to predict, find adj R2

x = df_sc_drop_2.drop(['production'], axis=1)
y = df_sc_drop_2['production']
                
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:             production   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     98.10
Date:                Tue, 27 Oct 2020   Prob (F-statistic):          6.51e-159
Time:                        03:48:44   Log-Likelihood:                 848.18
No. Observations:                1000   AIC:                            -1670.
Df Residuals:                     987   BIC:                            -1607.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0

In [178]:
######################
# as observed delete three attributes, the adj R2 does not significantly change
# the p value <0.05, the prediction model so far is well-built

################################################# again, use remaining varables to predict, find adj R2

############################## drop atrributes

df_sc_drop_3 = df_sc_drop_2.drop([#'tvd (ft)',
                                 #'well spacing',
                                #'log permeability',
                                #'porosity',
                                #'poisson ratio',
                                #'p-velocity',
                                #'s-velocity',
                                #'youngs modulus',
                                 #'total number of stages',
                                #'proppant volume',
                                 #'date_sec'#,
                                 #'op_number'
                                    ], axis=1)

###############################  again, use remaining varables to predict, find adj R2

x = df_sc_drop_3.drop(['production'], axis=1)
y = df_sc_drop_3['production']
                
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())
#### result, every variable is important

                            OLS Regression Results                            
Dep. Variable:             production   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     98.10
Date:                Tue, 27 Oct 2020   Prob (F-statistic):          6.51e-159
Time:                        03:48:47   Log-Likelihood:                 848.18
No. Observations:                1000   AIC:                            -1670.
Df Residuals:                     987   BIC:                            -1607.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0

In [170]:
#### result, every variable is important

In [171]:
## record the attributes and use the orginal data in the attributes to repeat the adj R 2

In [179]:
df_att = df[['tvd (ft)',
             'well spacing',
             'log permeability',
             'porosity',
             'poisson ratio',
             'p-velocity',
             's-velocity', 
             'youngs modulus', 
             'total number of stages',
             'proppant volume',
             'date_sec',
              'op_number',
             'production']]
df_att_drop = df_att.dropna()

###############################  again, use remaining varables to predict, find adj R2

x = df_att_drop.drop(['production'], axis=1)
y = df_att_drop['production']
                
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())
#### result, every variable is important

                            OLS Regression Results                            
Dep. Variable:             production   R-squared:                       0.549
Model:                            OLS   Adj. R-squared:                  0.542
Method:                 Least Squares   F-statistic:                     80.30
Date:                Tue, 27 Oct 2020   Prob (F-statistic):          1.28e-117
Time:                        03:49:07   Log-Likelihood:                -6114.2
No. Observations:                 739   AIC:                         1.225e+04
Df Residuals:                     727   BIC:                         1.231e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   7868

In [180]:
################################# correlation ##########################################

## the coefficient returns a value between -1 and 1 
## the value represents the limits of correlation from a full negative correlation to a full positive correlation
## according to reference, |value| <0.1 means weak correlation
## to be conservative, we choose |value| <0.01

df_att_drop_corr = df_att_drop.corr(method ='pearson').round(2) 
df_att_drop_corr

# list the correlation between featurs and production was in the range of -0.2 to 0.2

,tvd (ft),log permeability,porosity,poisson ratio,p-velocity,s-velocity,youngs modulus,total number of stages,proppant volume,date_sec,op_number,production
tvd (ft),1.00,0.17,0.27,-0.20,-0.45,0.14,0.11,-0.08,-0.13,-0.10,0.18,0.19
log permeability,0.17,1.00,0.72,-0.39,-0.44,-0.18,-0.31,-0.05,-0.05,-0.03,0.00,0.03
porosity,0.27,0.72,1.00,-0.51,-0.52,-0.10,-0.34,-0.11,-0.09,-0.14,-0.06,0.03
poisson ratio,-0.20,-0.39,-0.51,1.00,0.64,0.09,0.71,0.27,0.16,0.21,-0.01,-0.14
p-velocity,-0.45,-0.44,-0.52,0.64,1.00,0.20,0.43,0.17,0.15,0.17,-0.11,-0.25
s-velocity,0.14,-0.18,-0.10,0.09,0.20,1.00,0.63,-0.04,-0.09,-0.12,0.16,-0.21
youngs modulus,0.11,-0.31,-0.34,0.71,0.43,0.63,1.00,0.16,0.02,0.05,0.10,-0.24
total number of stages,-0.08,-0.05,-0.11,0.27,0.17,-0.04,0.16,1.00,0.79,0.64,-0.18,0.49
proppant volume,-0.13,-0.05,-0.09,0.16,0.15,-0.09,0.02,0.79,1.00,0.64,-0.16,0.58
date_sec,-0.10,-0.03,-0.14,0.21,0.17,-0.12,0.05,0.64,0.64,1.00,-0.14,0.42


In [181]:
################################# confirm the critical variables ##########################################
## after check with the original dataset
## we confirm that the critical variables are 
                                            #   'tvd (ft)', 
                                            #   'well spacing',
                                            #   'log permeability',            
                                            #   'porosity',            
                                            #   'poisson ratio',           
                                            #   'p-velocity',          
                                            #   's-velocity',             
                                            #   'youngs modulus',             
                                            #   'total number of stages',           
                                            #   'proppant volume',           
                                            #   'date_sec',          
                                            #   'op_number'            


In [175]:
############################################################################################################################
###############################                     find coefficient            ##########################################
############################################################################################################################